In [3]:
from Preprocess import *
from Model import *
from FeatureEng import *

ModuleNotFoundError: No module named &#39;numpuy&#39;

# Account
- account_id
- district_id
- date
- frequency

# Client
- client_id
- birth_number
- district_id

# Disposition
- disp_id
- client_id
- account_id
- type

# Loan
- loan_id
- account_id
- date
- amount
- duration
- payments
- status

# Transition
- trans_id
- account_id
- date
- type
- operation
- amount_balance
- k_symbol
- bank
- account

In [ ]:
dic_csv = read_data()
df_var = make_dataframe(dic_csv)



In [5]:
df_var

{&#39;client&#39;:       client_id  birth_number  district_id
 0             1        706213           18
 1             2        450204            1
 2             3        406009            1
 3             4        561201            5
 4             5        605703            5
 ...         ...           ...          ...
 5364      13955        456030            1
 5365      13956        430406            1
 5366      13968        680413           61
 5367      13971        626019           67
 5368      13998        535812           74
 
 [5369 rows x 3 columns],
 &#39;account&#39;:       account_id  district_id         frequency    date
 0            576           55  monthly issuance  930101
 1           3818           74  monthly issuance  930101
 2            704           55  monthly issuance  930101
 3           2378           16  monthly issuance  930101
 4           2632           24  monthly issuance  930102
 ...          ...          ...               ...     ...
 4495   